# Running a CAT based on a synthetic correct/incorrect MIRT model

This example shows how to run a CAT based on a synthetic correct/incorrect
MIRT model.

Import order is important. We put ComputerAdaptiveTesting last so we get the
extra dependencies.

In [1]:
using Makie
import Pkg
import Random
using Distributions: Normal, cdf
using AlgebraOfGraphics
using ComputerAdaptiveTesting
using ComputerAdaptiveTesting.ExtraDistributions: NormalScaledLogistic
using ComputerAdaptiveTesting.Sim: auto_responder
using ComputerAdaptiveTesting.NextItemRules: DRuleItemCriterion
using ComputerAdaptiveTesting.TerminationConditions: FixedItemsTerminationCondition
using ComputerAdaptiveTesting.Aggregators: PriorAbilityEstimator, MeanAbilityEstimator, LikelihoodAbilityEstimator
using ComputerAdaptiveTesting.ItemBanks
using ComputerAdaptiveTesting.Integrators
import ComputerAdaptiveTesting.IntegralCoeffs
using CATPlots

@automakie()

Now we are read to generate our synthetic data using the supplied DummyData
module. We generate an item bank with 100 items and fake responses for 3
testees.

In [2]:
dims = 2
using ComputerAdaptiveTesting.DummyData: dummy_full, std_mv_normal, SimpleItemBankSpec, StdModel4PL
using ComputerAdaptiveTesting.MathTraits
using ComputerAdaptiveTesting.Responses: BooleanResponse

TODO: pass in dims

In [3]:
(item_bank, question_labels, abilities, responses) = dummy_full(
    Random.default_rng(42),
    SimpleItemBankSpec(StdModel4PL(), VectorContinuousDomain(), BooleanResponse()),
    dims;
    num_questions=10,
    num_testees=2
)

(ComputerAdaptiveTesting.ItemBanks.SlipItemBank{ComputerAdaptiveTesting.ItemBanks.GuessItemBank{ComputerAdaptiveTesting.ItemBanks.CdfMirtItemBank{Distributions.Logistic{Float64}}}}([0.15985979921475818, 0.05776653910283313, 0.0, 0.12491046639792057, 0.0, 0.0, 0.16039115407259674, 0.14939183784418641, 0.24994479030198857, 0.15838910674738643], ComputerAdaptiveTesting.ItemBanks.GuessItemBank{ComputerAdaptiveTesting.ItemBanks.CdfMirtItemBank{Distributions.Logistic{Float64}}}([0.29719187358604565, 0.0, 0.0, 0.0, 0.0, 0.23911909050392627, 0.32090443034096733, 0.0, 0.0, 0.0], ComputerAdaptiveTesting.ItemBanks.CdfMirtItemBank{Distributions.Logistic{Float64}}(Distributions.Logistic{Float64}(μ=0.0, θ=0.5875440658049353), [-0.9104324071228524, -0.5692091009934306, -1.1279068329115067, 0.3893049817317737, -0.9397053810165131, 1.190671565315947, -2.1611673459798344, -0.8258366987771657, 0.06413894067866721, -0.4574389364112896], [0.7596712416978716 1.003521659022198 … 0.7922357224775933 1.12058334

Simulate a CAT for each testee and record it using CatRecorder.
CatRecorder collects information which can be used to draw different types of plots.

In [4]:
max_questions = 9
integrator = CubaIntegrator([-6.0, -6.0], [6.0, 6.0], CubaVegas(); rtol=1e-2)
ability_estimator = MeanAbilityEstimator(PriorAbilityEstimator(std_mv_normal(dims)), integrator)
rules = CatRules(
    ability_estimator,
    DRuleItemCriterion(ability_estimator),
    FixedItemsTerminationCondition(max_questions)
)

CatRules
  next_item: ComputerAdaptiveTesting.NextItemRules.ItemStrategyNextItemRule{ComputerAdaptiveTesting.NextItemRules.ExhaustiveSearch1Ply, ComputerAdaptiveTesting.NextItemRules.DRuleItemCriterion{ComputerAdaptiveTesting.Aggregators.MeanAbilityEstimator{ComputerAdaptiveTesting.Aggregators.PriorAbilityEstimator{Distributions.ZeroMeanIsoNormal{Tuple{Base.OneTo{Int64}}}}, ComputerAdaptiveTesting.Aggregators.FunctionIntegrator{ComputerAdaptiveTesting.Integrators.CubaIntegrator{ComputerAdaptiveTesting.Integrators.CubaVegas, Base.Pairs{Symbol, Float64, Tuple{Symbol}, NamedTuple{(:rtol,), Tuple{Float64}}}}}}}}
  termination_condition: ComputerAdaptiveTesting.TerminationConditions.FixedItemsTerminationCondition
  ability_estimator: ComputerAdaptiveTesting.Aggregators.MeanAbilityEstimator{ComputerAdaptiveTesting.Aggregators.PriorAbilityEstimator{Distributions.ZeroMeanIsoNormal{Tuple{Base.OneTo{Int64}}}}, ComputerAdaptiveTesting.Aggregators.FunctionIntegrator{ComputerAdaptiveTesting.Integra

XXX: We shouldn't need to specify xs here since the distributions are not used -- rework

In [5]:
points = 3
xs = repeat(range(-2.5, 2.5, length=points)', dims, 1)
raw_estimator = LikelihoodAbilityEstimator()
recorder = CatRecorder(xs, responses, integrator, raw_estimator, ability_estimator, abilities)
for testee_idx in axes(responses, 2)
    @debug "Running for testee" testee_idx
    tracked_responses, θ = run_cat(
        CatLoopConfig(
            rules=rules,
            get_response=auto_responder(@view responses[:, testee_idx]),
            new_response_callback=(tracked_responses, terminating) -> recorder(tracked_responses, testee_idx, terminating),
        ),
        item_bank
    )
    true_θ = abilities[:, testee_idx]
    abs_err = sum(abs.(θ .- true_θ))
    @info "convergence" true_θ θ abs_err
end

┌ Info: convergence
│   true_θ =
│    2-element Vector{Float64}:
│     -1.0953633562092246
│      0.03348099200195957
│   θ =
│    2-element Vector{Float64}:
│     -0.46718131986496453
│     -0.4736832739486242
└   abs_err = 1.1353463022948438
┌ Info: convergence
│   true_θ =
│    2-element Vector{Float64}:
│      1.157144525872549
│     -0.6472271240703346
│   θ =
│    2-element Vector{Float64}:
│     0.12475791951438912
│     0.5880269192887828
└   abs_err = 2.2676406497172774


Make a plot showing how the estimated value converges during the CAT.

In [6]:
conv_lines_fig = ability_convergence_lines(recorder; abilities=abilities)
conv_lines_fig

FigureGrid()

---

*This notebook was generated using [Literate.jl](https://github.com/fredrikekre/Literate.jl).*